In [1]:
import csv
from sklearn import preprocessing
import numpy as np
import pandas as pd

In [2]:
#Load Data
rootDir = r'C:\Users\soursinha\Documents\Sourabh\Python\Techgig\Code Gladiators-Invesco-DataSet'
activityFile = rootDir+ r'\Code-Gladiators-Activity.csv'
aumFile = rootDir + r'\Code-Gladiators-AUM.csv'
ieFile = rootDir + r'\Code-Gladiators-InvestmentExperience.csv'
transactFile = rootDir + r'\Code-Gladiators-Transaction.csv'

def load_data(file):
    with open(file,'rt') as csvfile:
        linereader= csv.reader(csvfile,delimiter=",",quotechar = "\"")
        lineNum =1
        transformData =[]
        for row in linereader:
            if lineNum == 1:
                header = row
            else:
                transformData.append(row)
            lineNum += 1
    return header,transformData

activityHeader,activityData = load_data(activityFile)
aumHeader,aumData = load_data(aumFile)
ieHeader,ieData = load_data(ieFile)
transactHeader,transactData = load_data(transactFile)

           
        

In [3]:
#Replace space with underscore
def replace_space(inArray):
    outArray=[]
    for val in inArray:
        val = val.replace(" ", "_")
        outArray.append(val)
    return outArray

ieHeader= replace_space(ieHeader)


In [4]:
dfActivity = pd.DataFrame(activityData,columns=activityHeader)
dfAUM = pd.DataFrame(aumData,columns=aumHeader)
dfIE = pd.DataFrame(ieData,columns=ieHeader)
dfTrans = pd.DataFrame(transactData,columns=transactHeader)

#print(dfActivity.head(),"\n",dfAUM.head(),"\n",dfIE.head(),"\n",dfTrans.head())


In [5]:
dfActivity.head()

,Unique_Advisor_Id,Month,Activity_Type,Activity_Count
0,1000103,2015 / 01,11,3
1,1000103,2015 / 02,22,3
2,1000103,2015 / 06,11,3
3,1000103,2015 / 06,17,3
4,1000103,2015 / 06,26,3


In [6]:
dfAUM.head()

,Unique_Advisor_Id,Unique_Investment_Id,Month,Shares,AUM
0,1000103,14142,2016 / 01,11648.1266666667,200464.26
1,1000103,14142,2016 / 02,4286.61333333333,74587.072
2,1000103,14142,2016 / 03,858.36,15819.5748333333
3,1000103,14142,2016 / 04,858.36,16016.9976666667
4,1000103,14142,2016 / 05,858.36,16059.9156666667


In [7]:
dfIE.head()

,Unique_Investment_Id,Morningstar_Category,Month,Investment,Rating,1_Yr_%_Rank,3_Yr_%_Rank,5_Yr_%_Rank,10_Yr_%_Rank,1_Yr_Return,...,10_Yr_Return,1_Yr_Excess_Return_vs_Primary_Ix,3_Yr_Excess_Return_vs_Primary_Ix,5_Yr_Excess_Return_vs_Primary_Ix,10_Yr_Excess_Return_vs_Primary_Ix,1_Yr_Excess_Return_vs_Category_Ix,3_Yr_Excess_Return_vs_Category_Ix,5_Yr_Excess_Return_vs_Category_Ix,10_Yr_Excess_Return_vs_Category_Ix,Net_Flows
0,11517,US Fund China Region,2015 / 01,Invesco Greater China R5,2,88,73,82,,-1.266491,...,,-18.251687,-4.351661,-4.182195,,-19.598927,-3.062672,-2.218734,,-2785691
1,11517,US Fund China Region,2015 / 02,Invesco Greater China R5,2,88,71,80,,-5.398281,...,,-21.966066,-4.50024,-4.844217,,-24.440463,-3.732307,-2.717431,,-41248406
2,11517,US Fund China Region,2015 / 03,Invesco Greater China R5,2,82,73,80,,5.262,...,,-13.32578,-3.444401,-4.430941,,-18.736656,-3.835977,-2.660131,,-66946984
3,11517,US Fund China Region,2015 / 04,Invesco Greater China R5,2,45,74,61,,31.871233,...,,-0.618948,-2.451617,-3.17177,,-16.184466,-3.447366,-2.418043,,161312871
4,11517,US Fund China Region,2015 / 05,Invesco Greater China R5,2,49,71,67,,25.576027,...,,1.411188,-1.719549,-3.220432,,-10.645269,-2.863178,-1.740341,,192212800


In [8]:
dfTrans.head()

,Unique_Advisor_Id,Unique_Investment_Id,Month,Transaction_Type,Code_1,Code_2,Code_3,Code_4,Code_5,Amount
0,1000103,20058,2016 / 01,P,1,48,1,7,4,4678.66666666667
1,1000103,20058,2016 / 01,P,1,48,1,7,4,353.066666666667
2,1000103,20058,2016 / 01,P,1,48,1,7,4,1809.62666666667
3,1000103,20058,2016 / 02,P,1,48,1,7,4,4678.66666666667
4,1000103,20058,2016 / 02,P,1,48,1,7,4,353.066666666667


In [9]:
#Map for Morning star cateogry
mrgCat_Map ={}
for i,val in enumerate(dfIE.Morningstar_Category.unique()):
    mrgCat_Map[val] = i

#Map for Morning star cateogry
inv_Map ={}
for i,val in enumerate(dfIE.Investment.unique()):
    inv_Map[val] = i

#print(inv_Map)


In [10]:
#create date feature
def convertDateToFeature(date):
    #this function will be used to convert date into a numerical feature
    import time
    from datetime import datetime
    dt = datetime.strptime(date, '%Y / %m')
    timestamp = int(time.mktime(dt.timetuple()))
    return timestamp

#this block is TODO

In [11]:
a = '2016 / 02'
import time
from datetime import datetime
dt = datetime.strptime(a, '%Y / %m')
print(dt)
dfActivity

2016-02-01 00:00:00


,Unique_Advisor_Id,Month,Activity_Type,Activity_Count
0,1000103,2015 / 01,11,3
1,1000103,2015 / 02,22,3
2,1000103,2015 / 06,11,3
3,1000103,2015 / 06,17,3
4,1000103,2015 / 06,26,3
5,1000103,2015 / 09,13,3
6,1000103,2015 / 09,6,3
7,1000103,2015 / 10,22,3
8,1000103,2015 / 12,13,3
9,1000103,2016 / 02,26,6


In [12]:
a1 = dfActivity.Month
dfActivity.Month = a1.apply(convertDateToFeature)
#dfActivity

In [14]:
dfActivity.head()

,Unique_Advisor_Id,Month,Activity_Type,Activity_Count
0,1000103,1420050600,11,3
1,1000103,1422729000,22,3
2,1000103,1433097000,11,3
3,1000103,1433097000,17,3
4,1000103,1433097000,26,3
